In [1]:

import plotly.express as px
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import io
import os
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import csv

In [2]:
df = pd.read_json(os.path.join(os.getcwd(), "result.json"))
df["recording_date"] = pd.to_datetime(df["recording_date"]).dt.date

df_grouped = (
    df.groupby(["recording_date", "customer_sentiment"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)
df_grouped["total_calls"] = df_grouped[
    ["positive", "neutral", "negative", "unknown"]
].sum(axis=1)

df_melted = df_grouped.melt(
    id_vars=["recording_date", "total_calls"],
    value_vars=["positive", "neutral", "negative", "unknown"],
    var_name="Sentiment",
    value_name="Count",
)

In [3]:
import warnings
warnings.filterwarnings("ignore")

fig = px.bar(
    df_melted,
    x="recording_date",
    y="Count",
    color="Sentiment",
    barmode="group",
    title="Customer Sentiment Analysis by Day",
    labels={
        "recording_date": "Date",
    },
    text="Count",
    range_color={"black","blue","yellow"}
)

for date, total in zip(df_grouped["recording_date"], df_grouped["total_calls"]):
    fig.add_annotation(
        x=date,
        y=max(df_melted["Count"]),
        text=f"Total:{total}",
        showarrow=False,
        yshift=10,
        font=dict(size=10, color="Black"),
    )

fig.show()

#### Top 10 Catagories

In [4]:
import numpy as np
import json

df = pd.read_json(os.path.join(os.getcwd(), "result.json"))

def extract_categories(json_str):
    try:
        # Attempt to parse the JSON and return the 'categories' key
        parsed_json = json.loads(json_str)
        return str(parsed_json.get("product_interest", None))
    except (TypeError, json.JSONDecodeError):
        # Handle missing or invalid JSON
        return "Nothing"


df["categories"] = df["feature_extraction"].apply(extract_categories)

In [5]:
df = df.loc[(df['categories'] != 'Nothing')]
category_counts = df['categories'].value_counts()

top_categories = category_counts.head(10)

# Create a bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_categories.index,
    y=top_categories.values,
    #marker_color='blue'
))

# Customize layout
fig.update_layout(
    title='Total Calls Based on Top 10 Categories',
    xaxis_title='Categories',
    yaxis_title='Number of Calls',
    xaxis_tickangle=-45  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()


### Word Cloud Dashboard

In [6]:
df = pd.read_json(os.path.join(os.getcwd(), "result.json"))

df['categories'] = df['feature_extraction'].apply(extract_categories)

In [7]:
import pandas as pd
import plotly.graph_objects as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from io import BytesIO
import base64

words = df['categories'].to_list()
words = ' '.join(words)
words = words.replace('/',' ').replace('Nothing','')

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(words)

# Save the word cloud to a bytes object
img_buffer = BytesIO()
wordcloud.to_image().save(img_buffer, format='PNG')
img_buffer.seek(0)

# Encode the image in base64 to embed in Plotly
img_base64 = base64.b64encode(img_buffer.read()).decode('utf-8')

# Create a Plotly figure to display the image
fig = go.Figure()

# Add the image to the figure
fig.add_layout_image(
    dict(
        source=f"data:image/png;base64,{img_base64}",
        xref="paper", yref="paper",
        x=0.5, y=0.5,  # Position at the center
        sizex=1, sizey=1,  # Scale to fit
        xanchor="center", yanchor="middle"
    )
)

# Update the layout to remove axes and padding
fig.update_layout(
    width=800,
    height=400,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis=dict(visible=False),
    yaxis=dict(visible=False)
)

# Show the figure
fig.show()
